**Part 4: Large Language Model**


In [ ]:
import time, random, numpy as np, argparse, sys, re, os
from types import SimpleNamespace

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, f1_score, recall_score, accuracy_score

# change it with respect to the original model
from tokenizer import BertTokenizer
from bert import BertModel
from tqdm import tqdm
from classifier import BertSentClassifier

***4.1 The fine-tuned minBERT classifier***


Instantiate a BertSentClassifier model. Load the state dict from the checkpoint saved from the
previous question.

Load the best model checkpoint from the previous question. and instantiate a BertSentClassifier

In [ ]:
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
filepath = 'flexible-50-5e-06.pt'


saved = torch.load(filepath)
config = saved['model_config']
model = BertSentClassifier(config)
model.load_state_dict(saved['model'])
model = model.to(device)

Specifiy the batch of sentences

Very positive: "This movie was absolutely incredible! The acting was superb, the plot was engaging, and the visuals were breathtaking. I loved every minute of it!"

Less positive: "The movie was decent overall, but I felt that the pacing was a bit slow in some parts. It could have been better with tighter editing."

Slightly negative: "The movie had its moments, but I found some of the characters and dialogue to be a bit cliche. It lacked originality in certain aspects."

Very negative: "I was extremely disappointed with this movie. The plot was convoluted, the acting was wooden, and the special effects were lackluster. I would not recommend it."

Off topic "The chair in the office is of standard size and design, suitable for everyday use."